In [ ]:
import fnmatch
import json
import getpass
import os
import pathlib
import datetime
                    
from dask.distributed import LocalCluster, SSHCluster 
from laserfarm import Retiler, DataProcessing, GeotiffWriter, MacroPipeline
from laserfarm.remote_utils import get_wdclient, get_info_remote, list_remote

## Global Configuration

In [2]:
# Configurations

import fnmatch
import json
import getpass
import os
import pathlib
import datetime
                    
from dask.distributed import LocalCluster, SSHCluster 
from laserfarm import Retiler, DataProcessing, GeotiffWriter, MacroPipeline
from laserfarm.remote_utils import get_wdclient, get_info_remote, list_remote

conf_remote_path_root = pathlib.Path('/webdav')
conf_remote_path_ahn = pathlib.Path('/webdav/ahn')
conf_remote_path_retiled = pathlib.Path('/webdav/retiled/')
conf_remote_path_targets = pathlib.Path('/webdav/targets')
conf_local_tmp = pathlib.Path('/tmp')

param_hostname = 'https://lfw-ds001-i022.lifewatch.dev:32443'
param_login = 'laser_data'
param_password = 'zDoy0hNKOYAVd'

param_feature_name = 'perc_95_normalized_height'
param_validate_precision = '0.00001'
param_tile_mesh_size = '10.'
param_filter_type= 'select_equal'
param_attribute = 'raw_classification'
param_min_x = '-113107.81'
param_max_x = '398892.19'
param_min_y = '214783.87'
param_max_y = '726783.87'
param_n_tiles_side = '512'
param_apply_filter_value = '1'

conf_wd_opts = { 'webdav_hostname': param_hostname, 'webdav_login': param_login, 'webdav_password': param_password}

## Fetching Laz Files from remote WebDAV

In [3]:
# Fetch Laz Files Demo

laz_files = [f for f in list_remote(get_wdclient(conf_wd_opts), conf_remote_path_ahn.as_posix())
             if f.lower().endswith('.laz')]

## Retiling of big files into smaller tiles

In [5]:
# Retiling Demo

remote_path_retiled = str(conf_remote_path_retiled)

grid_retile = {
    'min_x': -113107.81,
    'max_x': 398892.19,
    'min_y': 214783.87,
    'max_y': 726783.87,
    'n_tiles_side': 512
}


retiling_input = {
    'setup_local_fs': {'tmp_folder': conf_local_tmp.as_posix()},
    'pullremote': conf_remote_path_ahn.as_posix(),
    'set_grid': grid_retile,
    'split_and_redistribute': {},
    'validate': {},
    'pushremote': conf_remote_path_retiled.as_posix(),
    'cleanlocalfs': {}
}

file = laz_files
retiler = Retiler(file).config(retiling_input).setup_webdav_client(conf_wd_opts)
retiler_output = retiler.run()

2022-04-11 12:50:15,215 -           laserfarm.pipeline_remote_data -       INFO - Input dir set to /tmp/pipeline_input
2022-04-11 12:50:15,216 -           laserfarm.pipeline_remote_data -       INFO - Output dir set to /tmp/pipeline_output
2022-04-11 12:50:15,219 -           laserfarm.pipeline_remote_data -       INFO - Pulling from WebDAV /webdav/ahn/C_19HZ2.LAZ ...
2022-04-11 12:50:16,041 -           laserfarm.pipeline_remote_data -       INFO - ... pulling completed.
2022-04-11 12:50:16,044 -                        laserfarm.retiler -       INFO - Setting up the target grid
2022-04-11 12:50:16,045 -                        laserfarm.retiler -       INFO - Splitting file /tmp/pipeline_input/C_19HZ2.LAZ with PDAL ...
2022-04-11 12:50:16,177 -                        laserfarm.retiler -       INFO - ... splitting completed.
2022-04-11 12:50:16,180 -                        laserfarm.retiler -       INFO - Redistributing files to tiles ...
2022-04-11 12:50:16,189 -                        l

## Fetching retilied files (tiles) from remote WebDAV

In [7]:
# Feature Extraction Demo

t = tiles

tile_mesh_size = 10.
features = ['perc_95_normalized_height']

grid_feature = {
    'min_x': -113107.81,
    'max_x': 398892.19,
    'min_y': 214783.87,
    'max_y': 726783.87,
    'n_tiles_side': 512
}

feature_extraction_input = {
    'setup_local_fs': {'tmp_folder': conf_local_tmp.as_posix()},
    'pullremote': conf_remote_path_retiled.as_posix(),
    'load': {'attributes': ['raw_classification']},
    'normalize': 1,
    'apply_filter': {
        'filter_type': 'select_equal', 
        'attribute': 'raw_classification',
        'value': [1, 6]#ground surface (2), water (9), buildings (6), artificial objects (26), vegetation (?), and unclassified (1)
    },
    'generate_targets': {
        'tile_mesh_size' : tile_mesh_size,
        'validate' : True,
        **grid_feature
    },
    'extract_features': {
        'feature_names': features,
        'volume_type': 'cell',
        'volume_size': tile_mesh_size
    },
    'export_targets': {
        'attributes': features,
        'multi_band_files': False
    },
    'pushremote': conf_remote_path_targets.as_posix(),
#     'cleanlocalfs': {}
}
idx = (t.split('_')[1:])
processing = DataProcessing(t, tile_index=idx).config(feature_extraction_input).setup_webdav_client(conf_wd_opts)
processing.run()

2022-04-11 12:50:31,969 -           laserfarm.pipeline_remote_data -       INFO - Input dir set to /tmp/pipeline_input
2022-04-11 12:50:31,970 -           laserfarm.pipeline_remote_data -       INFO - Output dir set to /tmp/pipeline_output
2022-04-11 12:50:31,972 -           laserfarm.pipeline_remote_data -       INFO - Pulling from WebDAV /webdav/retiled/tile_248_285 ...
2022-04-11 12:50:33,171 -           laserfarm.pipeline_remote_data -       INFO - ... pulling completed.
2022-04-11 12:50:33,172 -                laserfarm.data_processing -       INFO - Loading point cloud data ...
2022-04-11 12:50:33,173 -                laserfarm.data_processing -       INFO - ... loading /tmp/pipeline_input/tile_248_285/C_19HZ2_1.LAZ
2022-04-11 12:50:33,174 -                          pylas.lasreader -      ERROR - lazrs failed to decompress points: lazrs is not installed
2022-04-11 12:50:33,311 -                laserfarm.data_processing -       INFO - ... loading completed.
2022-04-11 12:50:33,312